<a href="https://colab.research.google.com/github/dvoils/neural-network-experiments/blob/main/ion_channels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Understanding Ion Channel Dynamics

## Introduction to Detailed Neuron Models

This section provides an in-depth look at how detailed ion channel dynamics affect the behavior of spiking neurons. Building upon the foundational Hodgkin-Huxley (HH) framework, we explore how additional ion channels—particularly A-type potassium currents—introduce new functional properties like spike delay and adaptation.

## Canonical Hodgkin-Huxley Model

The Hodgkin-Huxley model describes the dynamics of a neuron's membrane potential $u(t)$ through a set of ordinary differential equations (ODEs). It includes three main ionic currents:

* **Sodium current (Na⁺):**
  $I_{\text{Na}} = \bar{g}_{\text{Na}} m^3 h (u - E_{\text{Na}})$

* **Potassium current (K⁺):**
  $I_{\text{K}} = \bar{g}_{\text{K}} n^4 (u - E_{\text{K}})$

* **Leak current:**
  $I_{\text{L}} = \bar{g}_{\text{L}} (u - E_{\text{L}})$

Each gating variable $x \in \{m, h, n\}$ evolves as:

$\frac{dx}{dt} = \frac{x_\infty(u) - x}{\tau_x(u)}$

These equations define the voltage-dependent steady-state value $x_\infty(u)$ and the time constant $\tau_x(u)$, capturing the rate of channel activation or inactivation.

We also included the explicitly defined **persistent sodium current**:

$I_{\text{NaP}} = \bar{g}_{\text{NaP}} m (u - E_{\text{Na}})$

This current is notable for its lack of inactivation and its role in promoting sustained depolarization.

## Characterizing Ion Channel Behavior

The activation ($m_\infty$) and inactivation ($h_\infty$) properties of various ion channels can be described in terms of their voltage-dependent steady-state values and time constants ($\tau_m$, $\tau_h$). These functions determine how quickly and under what conditions ion channels open or close in response to changes in membrane potential. The qualitative behaviors of these channels can be compared based on their characteristic activation thresholds and kinetic timescales.

| Channel | $\tau_m$ (ms) | $\tau_h$ (ms) | Functional Role                            |
| ------- | ------------- | ------------- | ------------------------------------------ |
| I\_NaP  | \~2           | N/A           | Persistent excitation, promotes bursting   |
| I\_Na   | \~1           | \~10          | Fast spike initiation                      |
| I\_T    | \~5           | \~50          | Low-threshold activation, burst triggering |
| I\_A    | \~5           | \~10          | Delays firing, spike timing regulation     |
| I\_K2   | \~20          | \~1000        | Spike-frequency adaptation                 |

## The Functional Impact of A-type Potassium Currents ($I_A$)

The effect of the A-type potassium current on spike timing can be observed in scenarios where a constant depolarizing current is applied, yet the neuron does not fire immediately. Instead, there is a significant delay—on the order of 100 milliseconds—before the first spike occurs. This delay arises because the A-type current initially activates and counteracts the depolarizing input, and only after its inactivation variable decays sufficiently does the membrane potential rise to threshold, allowing the neuron to fire.

This delay is explained by the slow inactivation of the A-type current:

* The activation variable $m_A$ quickly turns on the outward current.
* The inactivation variable $h_A$ decays slowly over time.
* While $h_A$ is high, the outward potassium current counteracts depolarization.
* Once $h_A$ becomes sufficiently small, the membrane potential is free to rise and reach spike threshold.


# Low-Threshold Calcium Currents and Permeability-Based Modeling

Low-threshold calcium currents contribute additional complexity to neuronal signaling. These calcium channels allow the passage of $\text{Ca}^{2+}$ ions and are described not by conductance, but by permeability. The current is a function of the maximal permeability $\bar{p}_\text{Ca}$, and is modulated by activation $m$ and inactivation $h$ gating variables. Unlike sodium and potassium ions, calcium ions exist in much lower concentrations inside the cell. The resulting steep concentration gradient means that even small changes in channel opening can lead to significant influxes of $\text{Ca}^{2+}$. The intracellular calcium concentration $[\text{Ca}^{2+}]_i$ is therefore highly sensitive to current flow, introducing the potential for feedback mechanisms within the neuron.

## The Goldman-Hodgkin-Katz Equation and Nonlinear Driving Force

The relationship between calcium current and membrane voltage is governed by the Goldman-Hodgkin-Katz (GHK) equation. This formulation takes into account both the membrane voltage $u$ and the calcium concentration gradient across the membrane, producing a nonlinear current-voltage relationship. Although the text does not yet present a complete ODE formulation for calcium dynamics or the GHK current equation, these descriptions prepare the groundwork for models that include calcium influx and regulation. Once explicit mathematical expressions are introduced, these currents can be simulated alongside sodium and potassium channels in a unified model.

# Goldman-Hodgkin-Katz (GHK) Equation

Next is a **mathematically defined current** for calcium ions via the Goldman-Hodgkin-Katz equation, along with important physiological behavior tied to **low-threshold calcium channels (T-type)**, especially the phenomenon of **postinhibitory rebound**.


## Low-Threshold Calcium Current and the GHK Equation

Calcium ion currents, particularly those mediated by T-type calcium channels, play a significant role in shaping neuronal excitability and response to inhibition. Unlike sodium and potassium channels, which are typically described by conductances, calcium currents are better represented in terms of **permeability**. The current through calcium channels is described by the **Goldman-Hodgkin-Katz (GHK) current equation**, which accounts for both the membrane potential and the steep calcium ion concentration gradient across the membrane.

The GHK current for calcium is given by:

$$
I_x = \bar{p}_x \, m^p \, h^q \, \frac{z^2 F^2}{RT} \cdot \frac{u \cdot e^{-u z F / RT} \cdot [\text{Ca}^{2+}]_i - [\text{Ca}^{2+}]_e}{1 - e^{-u z F / RT}}
$$

Where:

* $\bar{p}_x$: maximum permeability for the ion species (here, calcium),
* $m$ and $h$: activation and inactivation gating variables, with exponents $p$ and $q$,
* $z = 2$: valence of calcium ions,
* $F$: Faraday's constant $= 9.648 \times 10^4 \, \text{C/mol}$,
* $R$: gas constant $= 8.314 \, \text{V·C·K}^{-1} \text{mol}^{-1}$,
* $T$: absolute temperature (in Kelvin),
* $u$: membrane potential (in volts),
* $[\text{Ca}^{2+}]_i$, $[\text{Ca}^{2+}]_e$: intracellular and extracellular calcium concentrations.

This equation models the calcium current $I_x$ as a nonlinear function of voltage $u$ and the calcium gradient across the membrane. Because intracellular calcium is tightly regulated and typically very low (compared to the extracellular level, which is often assumed to be 2.0 mM), even modest increases in permeability can lead to substantial inward currents.

## Inactivating Nature of the T-Type Calcium Current

The T-type calcium current $I_T$ is an **inactivating current**, meaning that after the membrane depolarizes, the channel closes and the current fades. This is governed by the inactivation gating variable $h$, which typically drops to zero near resting potential due to prior activity. Notably, $I_T$ is distinct from the fast sodium current of the Hodgkin-Huxley model. Whereas sodium channels are inactivated at rest because their activation gate $m$ is near zero, T-type calcium channels are inactivated because their inactivation gate $h$ is near zero. This key difference makes the T-type current dependent on **prior hyperpolarization** for recovery.

## Postinhibitory Rebound

A particularly important functional consequence of $I_T$ is its role in **postinhibitory rebound**. This phenomenon occurs when a neuron that has been hyperpolarized (e.g., by an inhibitory synaptic input) suddenly depolarizes and fires an action potential after the inhibition ends. The hyperpolarization **removes inactivation** of the T-type calcium channels by increasing $h$, and once the inhibition lifts, the channels can briefly open, allowing calcium influx that triggers a spike.

Mathematically, this behavior is captured by the interplay of gating variables in the GHK current. After hyperpolarization, $h$ increases gradually; upon return to a more depolarized potential, $m$ quickly activates while $h$ is still elevated, leading to a transient inward current that depolarizes the membrane to threshold.

The time course of this rebound response depends heavily on the inactivation time constant of $h$. A long time constant means the T-type channels remain available just long enough to produce the rebound depolarization before inactivating again.
